# Random Forest


In [1]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
from dataframe_builder import DataFrameBuilder
# Ignorar Warning de sklearn (no es importante)
import warnings
warnings.filterwarnings('ignore',
                        message='Changing the shape of non-C contiguous array')


print "Creando dataframe..."


builder = DataFrameBuilder()
df = builder.build()

print "Listo"

Creando dataframe...
Listo


Saco la columna de text porque ocupa mucho espacio

# Optimización de hiperparámetros

Busquemos los mejores (posibles) hiperparámetros

Para eso, primero veamos qué hiperparámetros nos provee la implementación de SKLearn.

In [2]:
from sklearn.ensemble import RandomForestClassifier
help(RandomForestClassifier)

Help on class RandomForestClassifier in module sklearn.ensemble.forest:

class RandomForestClassifier(ForestClassifier)
 |  A random forest classifier.
 |  
 |  A random forest is a meta estimator that fits a number of decision tree
 |  classifiers on various sub-samples of the dataset and use averaging to
 |  improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is always the same as the original
 |  input sample size but the samples are drawn with replacement if
 |  `bootstrap=True` (default).
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimators : integer, optional (default=10)
 |      The number of trees in the forest.
 |  
 |  criterion : string, optional (default="gini")
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "entropy" for the information gain.
 |      Note: this parameter is tree-specific.
 |  
 |  max_features : int, fl



Opciones:

- n_estimators: cantidad de árboles
- criterion: gini o entropy. Son dos medidas distintas
- splitter: No estoy muy seguro
- max_depth: máxima profundidad del árbol
- max_features: qué porcentaje de variables tomo a la hora de partir un nodo.
  Estas variables se eligen aleatoriamente
- min_samples_split: cuántos elementos tengo que tener en un nodo para decidir
  partirlo

In [10]:
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, roc_auc_score

options = {
    'n_estimators': range(10, 100, 5),
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2, 52, 8),
    'max_features': np.arange(0.1, 1.0, 0.2),
    'min_samples_split': range(2, 102, 10),
}

# Preparo data para clasificar
X = df.design_matrix
y = df.outcomes

scoring_methods = [
    'precision', 
    'accuracy', 
    'f1',
    'recall', 
    'roc_auc'
]

for scoring in scoring_methods:
    clf = RandomForestClassifier()

    print("=" * 80 + "\n")
    print("Scoring {}".format(scoring))

    #search = GridSearchCV(clf, scoring=scoring, param_grid=options, n_jobs=4)
    search = RandomizedSearchCV(clf, scoring=scoring, param_distributions=options, n_jobs=4, n_iter=10)
    
    search.fit(X, y)
    
    print "Mejor combinación: {}".format(search.best_params_)
    print "Mejor valor: {}\n\n".format(search.best_score_)

    """
    Esto puede no tener mucho sentido. Estamos corriendo nuevamente cross_val_score... 
    """
    for other_scorer in [precision_score, accuracy_score, f1_score, recall_score, roc_auc_score]:
        y_pred = search.best_estimator_.predict(X)
        
        print "Valor para {} = {}".format(
            other_scorer.__name__,
            other_scorer(y, y_pred)
        )
        
    

    


Scoring precision
Mejor combinación: {'n_estimators': 15, 'min_samples_split': 2, 'criterion': 'entropy', 'max_depth': 26, 'max_features': 0.30000000000000004}
Mejor valor: 0.968245604638


Valor para precision_score = 0.993900632161
Valor para accuracy_score = 0.993913204519
Valor para f1_score = 0.993912903903
Valor para recall_score = 0.993925175948
Valor para roc_auc_score = 0.993913205258

Scoring accuracy
Mejor combinación: {'n_estimators': 70, 'min_samples_split': 22, 'criterion': 'entropy', 'max_depth': 26, 'max_features': 0.70000000000000007}
Mejor valor: 0.968294339157


Valor para precision_score = 0.983425414365
Valor para accuracy_score = 0.986171985925
Valor para f1_score = 0.986210293031
Valor para recall_score = 0.989010989011
Valor para roc_auc_score = 0.986172161172

Scoring f1
Mejor combinación: {'n_estimators': 70, 'min_samples_split': 12, 'criterion': 'entropy', 'max_depth': 50, 'max_features': 0.90000000000000013}
Mejor valor: 0.969289939464


Valor para precisio